## Intermediate submission results showcase

#### dataset preparation, mesh display and latent representation

In [1]:
import os
import torch
import numpy as np
import trimesh
from human_body_prior.tools.model_loader import load_model
from human_body_prior.models.vposer_model import VPoser
from human_body_prior.body_model.body_model import BodyModel
from human_body_prior.tools.omni_tools import copy2cpu as c2c


In [ ]:
#load from latent.pt
latents_dict = torch.load('vposer_latents.pt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
support_dir = 'VPoserModelFiles'
expr_dir = os.path.join(support_dir,'vposer_v2_05/')
bm_fname = os.path.join(support_dir,'smplx_neutral_model.npz')
amass_dir = 'AMASS_CMUsubset'

vp, ps = load_model(expr_dir, model_code=VPoser,
                              remove_words_in_model_weights='vp_model.',
                              disable_grad=True,
                              comp_device=device)
vp = vp.to(device)
bm = BodyModel(bm_fname=bm_fname).to(device)


'''
change below to see diff model or diff frame of some pose.
'''

subject = '01_01_poses'
fIdx = 140 #frame id, change to see different one

latent_vectors = latents_dict[subject].to(device)
amass_body_poZ = latent_vectors[fIdx:fIdx+1]

amass_body_pose_rec = vp.decode(amass_body_poZ)['pose_body'].contiguous().view(-1, 63)
recoveredPoses = {'pose_body':amass_body_pose_rec}
bmodelreco = bm(**recoveredPoses)
vreco = c2c(bmodelreco.v) #vertices of decoded one
faces = c2c(bm.f)

sample_amass_fname = os.path.join(amass_dir, subject + '.npz')
amass_body_pose = np.load(sample_amass_fname)['poses'][:, 3:66] #original.
amass_body_pose = torch.from_numpy(amass_body_pose).type(torch.float).to(device)
originalPoses = {'pose_body':amass_body_pose[fIdx:fIdx+1]}
bmodelorig = bm(**originalPoses)
vorig = c2c(bmodelorig.v)

verts = vorig[0] #original from amass
mesh1 = trimesh.base.Trimesh(verts, faces)
mesh1.visual.vertex_colors = [254, 254, 254]
verts = vreco[0] #after decoding from latents.pt
mesh2 = trimesh.base.Trimesh(verts, faces)
mesh2.visual.vertex_colors = [254, 66, 200]
mesh2.apply_translation([1, 0, 0]) #for comparison
meshes = [mesh1, mesh2]
trimesh.Scene(meshes).show()

/tmp/ipykernel_2090580/2004533484.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  latents_dict = torch.load('vposer_latents.pt')
/scratch/pbk5339/conda_envs/cse586_1/lib

#### MJPE error calculation (training free)